In [43]:
import csv
import pandas as pd
import re 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from collections import Counter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [39]:
df = pd.read_csv("uscities.csv")
us_city_names = []
for city in df["city"][:1000]:
    us_city_names.append(city)

In [3]:
docu_file = open("Factiva_trial.txt", "r")

In [4]:
x = docu_file.read()

In [5]:
article_list = x.strip().split("*AN*")
del article_list[-1]
year_list = []
for i in range(len(article_list)):
    contain_year = article_list[i]
    for element in contain_year.split("\n"):
        if '*PD*' in element:
            year = re.search(r"\d{4}", element).group(0)
            year_list.append(year)


In [6]:
data = pd.DataFrame({'Year':year_list, 'Content':article_list})

In [36]:
cleaned = []
city_list = []
for text in data['Content']:
    doc = []
    sent_list = text.split(".")
    for sent in sent_list:
        if "curfew" in sent:
            doc.append(sent)
    filtered = []
    for sample in doc:
        filtered.append(sample.replace("\n", " ").replace("\t", " ").replace("*",""))
    cleaned.append(filtered)
    main_bag = []
    for e in filtered:
        if '--' in e:
            bag = e.split('--')
            city = bag[0]
        else:
            city = ''
        main_bag.append(city)
    city_list.append(main_bag)

In [41]:
stated_city = []
for sentences in cleaned:
    for texts in sentences:
        words = texts.split(" ")
        span = []
        if words in us_city_names:
            span.append(us_city_names)
    stated_city.append(span)

In [42]:
print(stated_city)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [37]:
print(len(city_list))

22


In [38]:
table = pd.DataFrame({'Year':year_list, 'Curfew':cleaned, 'City':city_list})
table

,Year,Curfew,City
0,1997,[ weekday curfew on people under 18 in Fairban...,"[, , ]"
1,1997,[ New Smyrna Beach -- A dusk-to-dawn curfew im...,[ New Smyrna Beach ]
2,1997,"[ TD Alaska Anchorage -- More than 3,00...",[ TD Alaska Anchorage ]
3,1997,[ His plan ranges from imposing curfew on juve...,[]
4,1997,[ Marysville -- City Council voted to impose a...,[ Marysville ]
5,1997,[ Virginia Richmond -- ACLU lawyers are taki...,"[ Virginia Richmond , ]"
6,1997,[ Colorado Loveland -- City Council approved...,[ Colorado Loveland ]
7,1997,[ Pennsylvania Philadelphia -- Police and co...,[ Pennsylvania Philadelphia ]
8,1997,[ Parents of curfew violators could be fined u...,[]
9,1997,[ Virginia Manassas -- County officials on T...,"[ Virginia Manassas , ]"


In [31]:
for i in table['Curfew'][:10]:
    print(i)

[' weekday curfew on people under 18 in Fairbanks and North Pole', ' On weekends and in the summer the curfew would be 1 a', ' The curfew is similar to one adopted by Anchorage voters Nov']
[' New Smyrna Beach -- A dusk-to-dawn curfew imposed on portions of the community following a Sunday tornado will probably be in effect for several days, officials said']
["  TD      Alaska  Anchorage -- More than 3,000 minors have been cited for breaking the city's curfew, but officials say more than $226,000 in fines remains uncollected"]
[' His plan ranges from imposing curfew on juveniles on probation to creating a drug-free zone in Washington Square Park, long hallowed ground for dealers']
[' Marysville -- City Council voted to impose a curfew for teens']
['  Virginia  Richmond -- ACLU lawyers are taking the city of Charlottesville to court over a curfew for teen-agers', ' The curfew became effective March 1']
['  Colorado  Loveland -- City Council approved on first vote a curfew that bars yout

In [55]:
sample = {'New legislation was enacted in 1992, and a partnership was established between the Phoenix Police Department and the Department of Parks, Recreation, and Libraries (PRL)': '1', 'Chicago, Illinois Chicago passed its first curfew ordinance in July 1948': '1' , 'The organization already has taken on several curfew cases, including a 1995 Washington curfew and a San Diego curfew passed in 1947': '1', 'In June, the council passed a juvenile curfew ordinance modeled on one used in Dallas':'1' , 'In June, the council passed a juvenile curfew ordinance modeled on one used in DallasPasco commissioners passed a countywide curfew in March after more than 18 months of lobbying by Sheriff Lee Cannon When Zephyrhills passed its curfew ordinance in April, it was merely to modernize what had existed for some time, Police Chief Robert Howell said': '1',
         'Largo passed a curfew in February but granted a grace period to inform kids about it':'1', 'Moore and Oklahoma City passed curfew ordinances last month':'1', 'A curfew law was passed about a month ago in Pocatello':'1', 'The state judges decision probably invalidates most of the curfew laws on the books in at least two dozen Washington cities, including La Center, which enacted a curfew ordinance similar to Bellinghams 15 months ago':'1', 'Edmond City Attorney Steve Murdock said the current curfew ordinance was enacted in 1974':'1', 'South Charleston City Council enacted a curfew law in December 1993':'1', 'Pocatello and Blackfoot have recently passed similar ordinances':'1', 'Judge Emmet Sullivan said the District of Columbia Council had passed the curfew without credible statistical evidence that youths commit more crimes or become victims more often during the hours of the curfew: 11 P.M':'1',
         'Lakeland passed an ordinance last month requiring youths younger than 16 to be home by 11 p.m':'1', 'in November 1994, Pawtucket enacted a curfew that served as a model for the one the council approved last night':'1','The DeSoto County Commission unanimously passed an ordinance limiting the times juveniles are allowed to be on the streets, aligning itself with the city curfew enacted last month':'1', 'Largo passed a curfew in February modeled after Pinellas Parks':'1', 'Tampa passed a curfew in January, prohibiting people younger than 17 from being on the streets from 11 p.m':'1', 'Tampa has had a curfew for children under 17 since January 1994':'1', 'Largo adopted a similar ordinance in 1998':'1', 'Lakeland passed an ordinance last month requiring youths younger than 16 to be home by 11 p.m': '1', 'The DeSoto County Commission unanimously passed an ordinance limiting the times juveniles are allowed to be on the streets, aligning itself with the city curfew enacted last month':'1',
         'South Charleston City Council enacted a curfew law in December 1993':'1', 'the council passed a juvenile curfew ordinance modeled on one used in Dallas':'1', 'Joseph County Council gave unanimous approval Tuesday to a curfew for juveniles in the county areas outside South Bend and Mishawaka':'1'}
final_sample = {}
for key,value in sample.items():
    if key not in final_sample.keys():
        final_sample[key] = value

In [56]:
sample_zero = {'The proposed curfew would mirror one going into effect at 12:01 Saturday morning in Lakeland':'0', 'In Tampa, Mayor Sandy Freedman vetoed a juvenile curfew for Ybor City after the ACLU promised to sue if the curfew were enacted':'0','The Dallas curfew is a case in point':'0', 'The Denver curfew program enjoys a collaborative partnership with 234 community programs to which children and their families are diverted':'0', 'In support of the curfew ordinance, the Jacksonville Police Department, the Duval County Parks, Recreation, and Entertainment Department, and the Duval County School Board provide a range of community-based delinquency prevention programs':'0', '"We oppose the law because it violates the constitutional rights of teenagers," says Arthur Spitzer, legal director of the ACLUs Washington chapter':'0', 'The Tampa ordinance gave parents more leeway, he wrote, by allowing children to break curfew to run general errands for their parents, for example':'0', 
                'The highest courts in Iowa, Hawaii and Washington state have found curfews unconstitutional, while other courts have upheld curfews in Charlottesville, Va., and Dallas':'0', 'A 17-year-old South Bend teen received a ticket about 15 minutes past curfew on March 12 at the intersection of Cleveland Road and Riverside Drive':'0', 'The West Virginia justices - with Starcher dissenting - acknowledged that an ordinance in Charleston infringed on some civil liberties, but said the impact was not severe enough to be unconstitutional':'0', 'The thing is, Burke initially opposed the curfew in Largo':'0', 'In Dallas in 1995, the first full year of the curfew, the police picked up 4,000 young people, of whom 2,500 were repeat offenders who were given citations ordering them to court':'0',
              'He also ruled the city of Indianapolis policy of subjecting minors who break curfew to drug and alcohol tests violates the Fourth Amendment':'0', '"Juvenile crime has increased at a greater rate than adult crime in Dubuque in the 90s," Mauss said':'0','Pasco Sheriff Lee Cannon expects to reintroduce a proposed juvenile curfew to Pasco commissioners next week':'0', 'San Diego will continue to be one of the nations safest large cities. New York City had the lowest overall crime rate among big cities':'0',
              'Taylor thinks the curfew has been effective':'0','Kenton Tarver, East Lake High Editor: The one crime this curfew law wont stop is the robbery of memories, good times and friendship that, according to Tampa City Council, must end at 10 p.m':'0', 'Pedestrian killed on Providence Road BRANDON - A pedestrian died after being hit by a car Tuesday evening on Providence Road south of Brandon Boulevard':'0', '"The word has gotten out to the kids, stay off the streets after hours." Since the curfew took effect, Lakeland police have reported a 41 percent overall reduction in juvenile crime':'0', 'Strauss , that the exemptions under the Dallas ordinance, which permitted juveniles to exercise their fundamental rights and remain in public, demonstrated that the ordinance was narrowly tailored to meet the citys legitimate objectives':'0',
              'Rice acknowledged that Newark already has a curfew, but he said that it has proved unenforceable, largely, he maintained, because parents were not held responsible for their childrens violations':'0', 'A curfew in Largo also could be affected, but officials said it was too early to tell':'0', 'So, many of those who want to race, or just pose as if they were racers, go to illegal street events like these, or to similar industrial areas in Ontario, Gardena and Huntington Beach':'0', 'Tampa passed a similar curfew two years ago, but for the most part it has gone unenforced because police and city attorneys believed it would not stand up under legal challenges':'0', 'He expects something similar to happen if Largo enacts its curfew':'0',
              'Oklahoma City also has a curfew rule, patterned after a similar ordinance in Dallas':'0', 'Fourteen years later, in 1989, Simbi Waters challenged a juvenile curfew ordinance in the District of Columbia on the grounds that it violated her first, fourth, and fifth amendment rights':'0', 'With the Powder Springs City Councils adoption this week of a teen curfew':'0', 'In Tampa, Mayor Sandy Freedman vetoed a juvenile curfew for Ybor City after the ACLU promised to sue if the curfew were enacted':'0', 'Annie Brown Kennedy of Winston-Salem, N.C., the first black woman to serve in the North Carolina Legislature, announced that shell retire when her term ends in 1994':'0', '5841 Roswell Road: The 16-year-old girl had passed out in the Sandy Springs bar and an ambulance was called':'0'}
final_sample_zero = {}
for key,value in sample_zero.items():
    if key not in final_sample_zero.keys():
        final_sample_zero[key] = value

In [57]:
final_sample.update(final_sample_zero)

In [58]:
raw_feature = []
labels = []
feature_set = []
for key, value in final_sample.items():
    raw_feature.append(key.title())
    words = key.title().split(' ')
    data = []
    feature_set.append(data)
    labels.append(value)

In [66]:
x_train, x_test, y_train, y_test = tts(raw_feature, labels, test_size=0.3)

In [67]:
pipeline = Pipeline([('vect', CountVectorizer(stop_words='english', token_pattern =r'\b[^\d\W]+\b', min_df = 2, max_df = 0.5)),
                    ('clf', MultinomialNB())])

In [68]:
model = pipeline.fit(x_train, y_train)

In [69]:
print(str(model.score(x_test, y_test)))

0.625


In [ ]:
hits = []
for sents in table['Curfew']:
    list_sents = [sents]
    result = model.predict(list_sents)
    print(result[0])